In [1]:
import sys
sys.path.append('/home/g201381710/ExperimentsGH/DotlessArabic')
sys.path.append('/home/g201381710/.local/lib/python3.10/site-packages')
sys.path.append('/home/g201381710/anaconda3/envs/dl4/lib/python3.10/site-packages')

sys.path

['/home/g201381710/ExperimentsGH/DotlessArabic/dotless_arabic/analysis/notebooks',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/g201381710/.local/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages/IPython/extensions',
 '/home/g201381710/.ipython',
 '/home/g201381710/ExperimentsGH/DotlessArabic',
 '/home/g201381710/.local/lib/python3.10/site-packages',
 '/home/g201381710/anaconda3/envs/dl4/lib/python3.10/site-packages']

In [2]:
%cd ../../..

/home/g201381710/ExperimentsGH/DotlessArabic


In [3]:
import re,random,math
from collections import Counter
from functools import lru_cache

from farasa.segmenter import FarasaSegmenter

import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from scipy.stats import linregress

from dotless_arabic.processing import process,undot

from dotless_arabic.tokenizers import DisjointLetterTokenizer,FarasaMorphologicalTokenizer, WordTokenizer

from dotless_arabic.datasets.news.collect import collect_dataset_for_analysis as collect_news_dataset_for_analysis
from dotless_arabic.datasets.quran.collect import collect_dataset_for_analysis as collect_quran_dataset_for_analysis
from dotless_arabic.datasets.poems.collect import collect_dataset_for_analysis as collect_poems_dataset_for_analysis
from dotless_arabic.datasets.wikipedia.collect import collect_dataset_for_analysis as collect_wikipedia_dataset_for_analysis
from dotless_arabic.datasets.aggregated.collect import collect_dataset_for_analysis as collect_aggregated_dataset_for_analysis
from dotless_arabic.datasets.sanadset_hadeeth.collect import collect_dataset_for_analysis as collect_sanadset_hadeeth_dataset_for_analysis

In [4]:
random.seed(42)

In [5]:
# tokenizer = WordTokenizer()

In [6]:
@lru_cache()
def tokenize_dataset(dataset,tokenizer):
    tokenized_dataset = list()
    if isinstance(tokenizer,FarasaMorphologicalTokenizer):
        segmenter = FarasaSegmenter(interactive=True)
    for document in tqdm(dataset):
        if isinstance(tokenizer,FarasaMorphologicalTokenizer):
            tokenized_document = " ".join(
                tokenizer.split_text(
                    document,
                    segmenter=segmenter,
                )
            )
        else:
            tokenized_document = " ".join(tokenizer.split_text(document))
        tokenized_document = tokenized_document.replace("<##>", "")
        tokenized_document = re.sub("\s+", " ", tokenized_document)
        tokenized_dataset.append(tokenized_document)
    return tokenized_dataset

In [7]:
@lru_cache()
def words_counter(dataset,use_tqdm=True):
    if not use_tqdm:
        return Counter(word for item in dataset for word in item.split())    
    return Counter(word for item in tqdm(dataset) for word in item.split())
@lru_cache()
def undot_dataset(dataset):
    return list(map(undot, tqdm(dataset)))

@lru_cache()
def process_dataset(dataset):
    return list(map(process, tqdm(dataset)))

In [22]:
tokenized_datasets = {}

In [24]:
# tokenize datasets
tokenized_datasets[WordTokenizer.__name__] = [
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_quran_dataset_for_analysis()),tokenizer=WordTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_sanadset_hadeeth_dataset_for_analysis()),tokenizer=WordTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_poems_dataset_for_analysis()),tokenizer=WordTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_wikipedia_dataset_for_analysis()),tokenizer=WordTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_news_dataset_for_analysis()),tokenizer=WordTokenizer()))),
    ]

  0%|          | 0/6236 [00:00<?, ?it/s]

####################################################################################################
Original Number of Samples:
650,986
####################################################################################################
####################################################################################################
Number of Samples after dropping duplicates:
637,565
####################################################################################################


  0%|          | 0/637565 [00:00<?, ?it/s]

Using custom data configuration MagedSaeed--ashaar-719bb58a76ea0092
Found cached dataset parquet (/home/g201381710/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/7714858 [00:00<?, ?it/s]

  0%|          | 0/4636663 [00:00<?, ?it/s]

  0%|          | 0/2784041 [00:00<?, ?it/s]

In [25]:
# tokenize datasets
tokenized_datasets[FarasaMorphologicalTokenizer.__name__] = [
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_quran_dataset_for_analysis()),tokenizer=FarasaMorphologicalTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_sanadset_hadeeth_dataset_for_analysis()),tokenizer=FarasaMorphologicalTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_poems_dataset_for_analysis()),tokenizer=FarasaMorphologicalTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_wikipedia_dataset_for_analysis()),tokenizer=FarasaMorphologicalTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_news_dataset_for_analysis()),tokenizer=FarasaMorphologicalTokenizer()))),
        
    ]

[2023-03-09 09:18:48,262 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2023-03-09 09:18:50,023 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


  0%|          | 0/6236 [00:00<?, ?it/s]

####################################################################################################
Original Number of Samples:
650,986
####################################################################################################
####################################################################################################
Number of Samples after dropping duplicates:
637,565
####################################################################################################


[2023-03-09 09:19:11,165 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2023-03-09 09:19:12,950 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


  0%|          | 0/637565 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# tokenize datasets
tokenized_datasets[DisjointLetterTokenizer.__name__] = [
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_quran_dataset_for_analysis()),tokenizer=DisjointLetterTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_sanadset_hadeeth_dataset_for_analysis()),tokenizer=DisjointLetterTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_poems_dataset_for_analysis()),tokenizer=DisjointLetterTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_wikipedia_dataset_for_analysis()),tokenizer=DisjointLetterTokenizer()))),
        process_dataset(dataset=tuple(tokenize_dataset(dataset=tuple(collect_news_dataset_for_analysis()),tokenizer=DisjointLetterTokenizer()))),
    ]

In [15]:
for tokenizer in tqdm([WordTokenizer,FarasaMorphologicalTokenizer,DisjointLetterTokenizer]):
    aggregated_dataset = []
    for _dataset in tokenized_datasets[tokenizer.__name__]:
        aggregated_dataset.extend(_dataset)
    tokenized_datasets.append(aggregated_dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

KeyError: 'WordTokenizer'

In [16]:
tokenized_datasets.keys()

dict_keys(['DisjointLetterTokenizer'])

In [ ]:
for dataset in tqdm(all_datasets):
    random.shuffle(dataset)

In [ ]:
all_datasets_names = [
    'quran',
    'sanadset hadeeth',
    'poems',
    'wikipedia',
    'news',
    'aggregated',
]

In [ ]:
# we may need to report the error on the regression line: MSE, MAE, etc.
@lru_cache()
def get_heaps_law_constants(dataset,get_points_list=False,clear_lru_cache=False):
  X,Y = [],[]
  vocabulary = set()
  number_of_tokens = 0
  for index in tqdm(range(len(dataset))):
    
    counter = words_counter(
        tuple([dataset[index]]),
        use_tqdm=False,
    )
    
    number_of_tokens += sum(counter.values())
    X.append(
        math.log(
            number_of_tokens,
             math.e, #base
          ),
      )
    
    vocabulary.update(counter.keys())
    Y.append(
        math.log(
            len(vocabulary),
             math.e, #base
          ),
      )
    
    if clear_lru_cache:
      words_counter.cache_clear()
    
  slope,intercept,*_= linregress(X,Y)

  b = slope
  k = math.e**intercept

  if get_points_list:
    return b,k,(X,Y)
  return b,k

In [ ]:
get_heaps_law_constants(dataset=all_datasets[0])

In [ ]:
def plot_heaps_multiple(figsize=(25,15)):

  unlog = lambda points: list(map(lambda point: math.e**point,points))

  tokenizers = (WordTokenizer,FarasaMorphologicalTokenizerDisjointLetterTokenizer,)

  figure, axis = plt.subplots(nrows=len(tokenizers), ncols=2, figsize=figsize,dpi=200,layout='constrained')
    
  plt.rc('font',size=FONT_SIZE)
  figure.supxlabel('N',fontsize=FONT_SIZE)
  figure.supylabel('V',fontsize=FONT_SIZE)

  for irow,tokenizer in enumerate(tokenizers):
      # The first plot is for Quran dataset
      beta,k,(X,Y) = get_heaps_law_constants(dataset=tuple(tokenized_datasets[tokenizer.__name__][0]),get_points_list=True)
      dataset_name = all_datasets_names[0]
        
      undotted_beta,undotted_k,(undotted_X,undotted_Y) = get_heaps_law_constants(dataset=tuple(tokenized_datasets[tokenizer.__name__][0]),get_points_list=True)
       
      axis[irow,0].plot(unlog(X),unlog(Y),label=f'{dataset_name} dataset (k={k:.3f},β={beta:.3f})')
      axis[irow,0].plot(
          unlog(X),
          [k*x**beta for x in unlog(X)],
          label=f'dotted regression fit for {dataset_name}',
          linestyle='dashed',
      )
    
      axis[irow,0].plot(unlog(undotted_X),unlog(undotted_Y),label=f'undotted {dataset_name} dataset (k={undotted_k:.3f},β={undotted_beta:.3f})')
      axis[irow,0].plot(
          unlog(undotted_X),
          [undotted_k*x**undotted_beta for x in unlog(undotted_X)],
          label=f'undotted regression fit for {dataset_name}',
          linestyle='dashed',
      )


     
      for i,dataset_name in enumerate(all_datasets_names[1:]):

          beta,k,(X,Y) = get_heaps_law_constants(dataset=tuple(tokenized_datasets[tokenizer.__name__][i+1]),get_points_list=True)
            
          undotted_beta,undotted_k,(undotted_X,undotted_Y) = get_heaps_law_constants(dataset=tuple(tokenized_datasets[tokenizer.__name__][i+1]),get_points_list=True)
           
          axis[irow,1].plot(unlog(X),unlog(Y),label=f'{dataset_name} dataset (k={k:.3f},β={beta:.3f})')
          axis[irow,1].plot(
              unlog(X),
              [k*x**beta for x in unlog(X)],
              label=f'dotted regression fit for {dataset_name}',
              linestyle='dashed',
          )
        
          axis[irow,1].plot(unlog(undotted_X),unlog(undotted_Y),label=f'undotted {dataset_name} dataset (k={undotted_k:.3f},β={undotted_beta:.3f})')
          axis[irow,1].plot(
              unlog(undotted_X),
              [undotted_k*x**undotted_beta for x in unlog(undotted_X)],
              label=f'undotted regression fit for {dataset_name}',
              linestyle='dashed',
          )  
    
      # plt.legend(loc='lower right',fontsize=14)
      # plt.ylim(ymin=0)
      # plt.xlim(xmin=0)
      # https://stackoverflow.com/questions/6390393/matplotlib-make-tick-labels-font-size-smaller
      # plt.xticks(fontsize=14)
      # plt.yticks(fontsize=14)
      # https://stackoverflow.com/questions/34227595/how-to-change-font-size-of-the-scientific-notation-in-matplotlib
      # https://stackoverflow.com/questions/21512305/inconsistent-font-size-for-scientific-notation-in-axis
      # plt.rc('font',size=12)

In [ ]:
plot_heaps_multiple()

In [ ]:
# plot_heaps(
#     datasets=[all_datasets[0]],
#     datasets_names=[all_datasets_names[0]],
# )

In [ ]:
# plot_heaps(
#     datasets=all_datasets[:0:-1],
#     datasets_names=all_datasets_names[:0:-1],
# )